In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
device = "cuda"  if torch.cuda.is_available() else "cpu"
import numpy as np
import pandas as pd


In [ ]:
X_train = np.load("/content/X_train_up.npy")
X_test = np.load("/content/X_test_up (1).npy")

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components =13)

X_train_tr = []

for i in range(len(X_train)):

    tt  =  pca.fit_transform(X_train[i])
    X_train_tr.append(tt)
X_train_tr = np.array(X_train_tr)

In [ ]:
X_test_tr = []

for i in range(len(X_test)):

    tt  =  pca.fit_transform(X_test[i])
    X_test_tr.append(tt)
X_test_tr = np.array(X_test_tr)

In [ ]:
X_train =  torch.tensor(X_train_tr,requires_grad= True).to(device)
X_test =  torch.tensor(X_test_tr,requires_grad = True).to(device)

In [ ]:
X_train.shape

torch.Size([224, 750, 13])

In [ ]:
y_train  = torch.load("/content/y_train.pt",map_location = device)
y_test = torch.load("/content/y_test.pt",map_location = device)


In [ ]:
for i in range(len(y_train)):
    y_train[i]-=1;
for i in range(len(y_test)):
    y_test[i]-=1;


In [ ]:
X_test.shape

torch.Size([56, 750, 13])

In [ ]:
batch_size=10
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=10, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=10,shuffle = True)


In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))


        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# for model 5 and 6
# input_size = 32
# hidden_size = 64
# num_layers = 6
# num_classes = 9
# learning_rate = 0.001

input_size = 13
hidden_size = 64
num_layers = 6
num_classes = 9
learning_rate = 0.001

model = LSTMClassifier(input_size,hidden_size,num_layers,num_classes).to(device)

In [ ]:
# model = LSTMClassifier(32,64,6,9)
# # model.state_dict()

# model.load_state_dict(torch.load('/content/model_5.pth'))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(train_loader):
        sequences = sequences.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels.squeeze())
        # print(loss)
        # Backward and optimize

        # loss.backward()
        loss.backward(retain_graph=True)
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')



Epoch [1/20], Step [10/23], Loss: 2.2214
Epoch [1/20], Step [20/23], Loss: 2.2036
Epoch [2/20], Step [10/23], Loss: 2.2337
Epoch [2/20], Step [20/23], Loss: 2.2142
Epoch [3/20], Step [10/23], Loss: 2.2075
Epoch [3/20], Step [20/23], Loss: 2.1901
Epoch [4/20], Step [10/23], Loss: 2.1476
Epoch [4/20], Step [20/23], Loss: 2.2318
Epoch [5/20], Step [10/23], Loss: 2.1508
Epoch [5/20], Step [20/23], Loss: 2.1429
Epoch [6/20], Step [10/23], Loss: 1.9912
Epoch [6/20], Step [20/23], Loss: 2.3442
Epoch [7/20], Step [10/23], Loss: 1.9490
Epoch [7/20], Step [20/23], Loss: 1.9673
Epoch [8/20], Step [10/23], Loss: 1.8060
Epoch [8/20], Step [20/23], Loss: 1.7175
Epoch [9/20], Step [10/23], Loss: 1.7491
Epoch [9/20], Step [20/23], Loss: 1.7698
Epoch [10/20], Step [10/23], Loss: 1.7069
Epoch [10/20], Step [20/23], Loss: 1.4630
Epoch [11/20], Step [10/23], Loss: 1.4942
Epoch [11/20], Step [20/23], Loss: 1.3870
Epoch [12/20], Step [10/23], Loss: 1.5385
Epoch [12/20], Step [20/23], Loss: 2.0334
Epoch [13/

KeyboardInterrupt: 

In [ ]:
train_acc = 0
with torch.inference_mode():
    for sequences,label in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        preds = model(sequences)

        preds =  torch.argmax(preds,axis =1)

        train_acc += (preds==label.squeeze()).sum()

print(f"accuracy on training set {train_acc/len(y_train)}")

In [ ]:
test_acc = 0
with torch.inference_mode():
    for sequences,label in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        preds = model(sequences)

        preds =  torch.argmax(preds,axis =1)

        test_acc += (preds==label.squeeze()).sum()

print(f"accuracy on testing set {test_acc/len(y_test)}")

In [ ]:
torch.save(model.state_dict(),"model_7_pca.pth")

In [ ]:
ya_preds = model(X_test)


In [ ]:
ya_preds = torch.argmax(ya_preds,axis =1)

In [ ]:
y_test.squeeze()

In [ ]:
from sklearn.metrics  import accuracy_score

accuracy_score(y_test.view(-1,1).squeeze().detach().numpy(),ya_preds.detach().cpu())


In [ ]:
# import pandas as pd
# test = pd.read_excel("/content/Test.xlsx")

# test = test.drop(columns= ["ID"])
# test = test.values

# test = torch.tensor(test,dtype = torch.float32)
# def create_data_pt(random_tensor: torch.Tensor)->torch.Tensor:

#     le =  int(random_tensor.shape[0]/32)

#     arr = torch.zeros(le,random_tensor.shape[1]-1,32).to(device)
#     for i in range(le):

#         data_pt = random_tensor[32*i: 32*(i+1),:-1]
#         data_pt = data_pt.T
# #         print(data_pt.shape)
#         arr[i] =  data_pt


#     return arr

# test_points=  create_data_pt(test)

# preds = model(test_points)
# preds  = torch.argmax(preds ,axis=1)
# preds = preds.numpy()



# dictt = {0: 'Tiger',
#     1: 'Snake',
#     2: 'Wolf',
#     3: 'Bear',
#     4: 'Rabbit',
#     5: 'Monkey',
#     6: 'Eagle',
#     7: 'Dolphin',
#     8: 'Koala'}

# arr1  = [ dictt[i]  for i in preds]
# arr1

In [ ]:
test_point = torch.load("/content/test_points.pt")
test_point.shape

torch.Size([271, 749, 32])

In [ ]:
test_points = []

for i in range(len(test_point)):

    tt  =  pca.fit_transform(test_point[i].detach().cpu())
    test_points.append(tt)
test_points_pca = np.array(test_points)


In [ ]:
test_point.shape

torch.Size([271, 749, 32])

In [ ]:
test_point = torch.tensor(test_points_pca,dtype = torch.float32).to(device)

In [ ]:
preds = model(test_point)
preds  = torch.argmax(preds ,axis=1)
preds = preds.numpy()



dictt = {0: 'Tiger',
    1: 'Snake',
    2: 'Wolf',
    3: 'Bear',
    4: 'Rabbit',
    5: 'Monkey',
    6: 'Eagle',
    7: 'Dolphin',
    8: 'Koala'}

arr1  = [ dictt[i]  for i in preds]
arr1

In [ ]:
torch.save(test_points,"test_points.pt")